# How do I make my pandas DataFrame smaller and faster?


In [1]:
import pandas as pd

In [2]:
drinks = pd.read_csv('http://bit.ly/drinksbycountry')

In [3]:
drinks.head()

,country,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol,continent
0,Afghanistan,0,0,0,0.0,Asia
1,Albania,89,132,54,4.9,Europe
2,Algeria,25,0,14,0.7,Africa
3,Andorra,245,138,312,12.4,Europe
4,Angola,217,57,45,5.9,Africa


In [4]:
drinks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193 entries, 0 to 192
Data columns (total 6 columns):
country                         193 non-null object
beer_servings                   193 non-null int64
spirit_servings                 193 non-null int64
wine_servings                   193 non-null int64
total_litres_of_pure_alcohol    193 non-null float64
continent                       193 non-null object
dtypes: float64(1), int64(3), object(2)
memory usage: 9.1+ KB


    "memory usage: 9.1+ KB" on the last line dennotes memory usage by the dataframe. "+" symbol is used as object columns are refrences to other object columns , Pandas wants this info method to runs fast so it avoids looking at refrenced object . It actually figures out how much space refrence to that object takes. Hence + denotes that it is atleast 9.1 kb but it might be more thant based on the objects

#### Counting true memory usage. 
_pandas can be forced to count true memory usage below is the example._

In [5]:
drinks.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193 entries, 0 to 192
Data columns (total 6 columns):
country                         193 non-null object
beer_servings                   193 non-null int64
spirit_servings                 193 non-null int64
wine_servings                   193 non-null int64
total_litres_of_pure_alcohol    193 non-null float64
continent                       193 non-null object
dtypes: float64(1), int64(3), object(2)
memory usage: 30.4 KB


###### How much memory each column is taking.

In [7]:
drinks.memory_usage()
#below numbers are in bytes

Index                             80
country                         1544
beer_servings                   1544
spirit_servings                 1544
wine_servings                   1544
total_litres_of_pure_alcohol    1544
continent                       1544
dtype: int64

In [8]:
drinks.memory_usage(deep=True)


Index                              80
country                         12588
beer_servings                    1544
spirit_servings                  1544
wine_servings                    1544
total_litres_of_pure_alcohol     1544
continent                       12332
dtype: int64

By default object columns are not checked ,similar to drinks.info() hence the secound option  where deep=True is used .

In [13]:
type(drinks.memory_usage(deep=True))

pandas.core.series.Series

As we can see drinks.memory usage is a series , we can use series method .sum() to get ovrall value . Below is the example.

In [10]:
drinks.memory_usage(deep=True).sum()

31176

#### Storing strings as integers.

As we can see objects/string values takes more memories and this can be changed by storing integers that represents string values.

###### sorting and getting unique value for a prticular series

In [14]:
sorted(drinks.continent.unique())

['Africa', 'Asia', 'Europe', 'North America', 'Oceania', 'South America']

We can see only 6 continents here. Now if we have to store data in integer format, a refrence column needs to be created which would be memory consuming and incase of more number of unique value it would be too much of manual work to write.

However pandas comes with an inbuilt option known as category. Below is the example.

In [15]:
drinks['continent']=drinks.continent.astype('category')

In [19]:
drinks.dtypes

country                           object
beer_servings                      int64
spirit_servings                    int64
wine_servings                      int64
total_litres_of_pure_alcohol     float64
continent                       category
dtype: object

We can see datatype for continent is now updated to category.

In [20]:
drinks.continent.head()

0      Asia
1    Europe
2    Africa
3    Europe
4    Africa
Name: continent, dtype: category
Categories (6, object): [Africa, Asia, Europe, North America, Oceania, South America]

Although contienent column is getting stored as category we can see they default representation stays same.However under the hood its storing object into integers . This can also be checked using below example.

In [21]:
drinks.continent.cat.codes.head()

0    1
1    2
2    0
3    2
4    0
dtype: int8

Diconstructing - drinks-DataFrame,continent-series , cat-similar to string method where str is used and then other arguments,incase of category .cat is used. 

Output above shows how pandas is now represnting the integers.

In [22]:
drinks.memory_usage(deep=True)

Index                              80
country                         12588
beer_servings                    1544
spirit_servings                  1544
wine_servings                    1544
total_litres_of_pure_alcohol     1544
continent                         744
dtype: int64

continent is now only 744 bytes compare to 12332 in the begining.

In [23]:
drinks['country']=drinks.country.astype('category')

In [24]:
drinks.memory_usage(deep=True)

Index                              80
country                         18094
beer_servings                    1544
spirit_servings                  1544
wine_servings                    1544
total_litres_of_pure_alcohol     1544
continent                         744
dtype: int64

Country rather than reduction of size has increase in size . The reason is every country is a different string . which leads to creation of 193 integers pointing to lookup table with 193 string.

NOTE: 
* Bottom line is category option should be used only when unique value is less.
* Converting a string/object field to category not only will save space but will also increase computation speed.

In [25]:
df = pd.DataFrame({'ID':[100,101,102,103],
                   'quality':['good','very good','good','excellent']})

In [26]:
df

,ID,quality
0,100,good
1,101,very good
2,102,good
3,103,excellent


In [27]:
df.sort_values('quality')

,ID,quality
3,103,excellent
0,100,good
2,102,good
1,101,very good


Above data frame provides quality for ID types , and  sorting is done based on alphabetic order. Incase sorting is required to sort as per logic.

In [28]:
df['quality']=df.quality.astype('category',categories=
                                ['good','very good','excellent'],ordered=True)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  


quality series is converted into category and then categories are added in their order of logic.

In [30]:
df.sort_values('quality')

,ID,quality
0,100,good
2,102,good
1,101,very good
3,103,excellent


In [31]:
df.quality

0         good
1    very good
2         good
3    excellent
Name: quality, dtype: category
Categories (3, object): [good < very good < excellent]

Post converting quality series into category and adding order of logic. Order of sorting by default has changed, also on selecting series it shows the logic. 

Boolean condition can be used for example if it is to be found  scenario where quality is better than good .

In [32]:
df.loc[df.quality>'good',:]

,ID,quality
1,101,very good
3,103,excellent
